In [ ]:
import matplotlib.patches
import numpy as np
import seaborn as sns
from IPython.display import display
from ipywidgets import interact
from jupyter_renderer_widget import PyplotRenderer
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator
from scipy.special import factorial

In [ ]:
def f(x):
    #return x ** 2 / 100
    return np.exp(np.sin(x) * 4)

x_vals = np.linspace(0, 10, 100)
y_vals = f(x_vals)

@interact(
    x1=(x_vals.min(), x_vals.max(), 0.1),
    h=(0.01, 5., 0.01),
)
def plot(
    x1=2.,
    h=4.,
):
    y1 = f(x1)    
    x2 = x1 + h
    y2 = f(x1 + h)
    slope = np.log(y2 / y1) / h
    #slope = (y2 - y1) / h
    #y3 = y2_prime * h + y1
    x4_vals = np.linspace(x1, x2, 50)
    y4_vals = np.exp((x4_vals - x1) * slope) * y1

    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    ax.plot(x_vals, y_vals)
    ax.plot(x1, y1, 'go')
    ax.plot(x2, y2, 'bo')
    #ax.plot(x2, y3, 'ro')
    #ax.plot([x1, x2], [y1, y3], 'r--')
    ax.plot(x4_vals, y4_vals, 'r--')

In [ ]:
@interact(
    max_x=(0.01, 100, 0.01),
)
def f(
    max_x=4,
):
    x_vals = np.linspace(0, max_x, 100)
    y_vals = np.exp(x_vals) / np.exp(max_x)
    _, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.plot(x_vals, y_vals)
    ax.set_xlim(0, max_x)
    ax.set_ylim(0, 1)
    ax.set_title('Exponential asymptotic behavior when zooming out horizontally')

In [ ]:
@interact(
    n=(1, 40, 1),
    x=(-4., 4., 0.01),
    y=(-8., 8., 0.01),
)
def f(
    n=25,
    x=0.,
    y=3.14,
    power_series=True,
    binom_series=False,
):
    xy = x + 1j * y
    terms1 = np.cumsum([xy ** i / factorial(i) for i in range(n)])
    terms2 = np.array([1] + [np.power(1 + xy / i, i) for i in range(1, n)])

    _, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.set_title('Exponential power & binomial series approximations')
    ax.set_xlim(-8., 8.)
    ax.set_ylim(-8., 8.)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', ls='--', lw=0.5)
    ax.axvline(0, color='k', ls='--', lw=0.5)
    ax.grid()
    ax.plot([0, x], [0, y], 'r-')
    ax.plot(x, y, 'r*')
    if power_series:
        ax.plot(np.real(terms1), np.imag(terms1), 'go-', ms=4.)
    if binom_series:
        ax.plot(np.real(terms2), np.imag(terms2), 'bo-', ms=4.)
    ax.plot(np.real(np.exp(xy)), np.imag(np.exp(xy)), 'ro')

In [ ]:
FRAMES = 250
GRID_SIZE_X = 6
GRID_SIZE_Y = 2 * np.pi
GRID_MAJOR = 60
GRID_MINOR = 5
GRID_COUNT = GRID_MAJOR * GRID_MINOR + 1
MIN_SCALE = 1.
MAX_SCALE = 1.

x_vals = np.linspace(-GRID_SIZE_X, GRID_SIZE_X, GRID_COUNT)
y_vals = np.linspace(-GRID_SIZE_Y, GRID_SIZE_Y, GRID_COUNT)
x_vals = np.where(np.isclose(x_vals, 0), np.zeros_like(x_vals), x_vals)
y_vals = np.where(np.isclose(y_vals, 0), np.zeros_like(y_vals), y_vals)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xy_grid = (x_grid + y_grid * 1j)
xy_vals = xy_grid.ravel()

def render(ax, frame_num):
    keyframe_1 = int(FRAMES * 0.00)
    keyframe0 = int(FRAMES * 0.04)
    keyframe1 = int(FRAMES * 0.46)
    keyframe2 = int(FRAMES * 0.54)
    keyframe3 = int(FRAMES * 0.96)
    keyframe4 = int(FRAMES * 1.00)
    keyframe0b = (keyframe1 - keyframe0) // 2 + keyframe0
    keyframe2b = (keyframe3 - keyframe2) // 2 + keyframe2
    if frame_num < keyframe0:
        t = 0.
    elif frame_num < keyframe0b:
        sub_t = (frame_num - keyframe0) / (keyframe0b - keyframe0)
        t = sub_t ** 2 / 2
    elif frame_num < keyframe1:
        sub_t = (frame_num - keyframe0b) / (keyframe1 - keyframe0b)
        t = 1 - (1. - sub_t) ** 2 / 2
    elif frame_num < keyframe2:
        t = 1.
    elif frame_num < keyframe2b:
        sub_t = (frame_num - keyframe2) / (keyframe2b - keyframe2)
        t = 1 - sub_t ** 2 / 2
    elif frame_num < keyframe3:
        sub_t = (frame_num - keyframe2b) / (keyframe3 - keyframe2b)
        t = (1. - sub_t) ** 2 / 2
    else:
        t = 0.

    #z_vals = xy_vals ** (1 + t)
    with np.errstate(all='ignore'):
        #z_grid = (1 - t) * xy_grid + t * np.exp(t * xy_grid + (1 - t) * np.log(xy_grid))
        z_grid = np.exp(t * xy_grid + (1 - t) * np.log(xy_grid))
        #z_grid = np.where(
        #    np.isclose(xy_grid.imag, 0),
        #    np.exp(t * xy_grid),  # + (1 - t) * xy_grid,
        #    t * xy_grid + (1 - t) * np.exp(t * xy_grid + (1 - t) * np.log(xy_grid)),
        #)
    #z_grid_interp = xy_grid * (1. - t) + z_grid * t
    z_grid_interp = z_grid
    z_vals_interp = z_grid_interp.ravel()
    ax.set_title('Exponential function on the complex plane: a+bi -> e^(a+bi)')
    #ax.set_title('X^4 transformation on complex plane')
    scale = MIN_SCALE
    if MAX_SCALE - MIN_SCALE > 0.:
        scale *= np.exp(np.log(MAX_SCALE / MIN_SCALE) * t)
    ax.set_xlim(-10. * scale, 10. * scale)
    ax.set_ylim(-10. * scale, 10. * scale)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', ls='--', lw=0.5)
    ax.axvline(0, color='k', ls='--', lw=0.5)
    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.yaxis.set_major_locator(MultipleLocator(10))
    if scale < 6:
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.grid()
    ax.set_xlabel('real (a)')
    ax.set_ylabel('imaginary (bi)')
    
    z_points = z_grid_interp[::GRID_MINOR, ::GRID_MINOR].ravel()
    ax.plot(z_points.real, z_points.imag, 'o', ms=2, alpha=0.6)
    for i in range(0, z_grid.shape[0], GRID_MINOR):
        vals = z_grid_interp[i, :]
        y = xy_grid[i, 0].imag
        alpha = 0.5
        lw = 0.6
        color = 'b'
        if np.isclose(y, 0):
            # Hack to handle bifurcation for x<0 y=0 due to log ambiguity.
            ax.plot(vals.real, -vals.imag, '-', color=color, lw=lw, alpha=alpha)
        elif np.isclose(y, np.pi):
            color = 'r'
            alpha = 1.
            lw = 1.5
        elif np.isclose(y, -np.pi):
            color = 'purple'
            alpha = 1.
            lw = 1.5
        ax.plot(vals.real, vals.imag, '-', color=color, lw=lw, alpha=alpha)

    for i in range(0, z_grid.shape[1], GRID_MINOR):
        vals = z_grid_interp[:, i]
        x = xy_grid[0, i].real
        alpha = 0.5
        lw = 0.6
        color = 'b'
        if np.isclose(x, 0.):
            color = 'lightgreen'
            alpha = 1.
            lw = 2
        ax.plot(vals.real, vals.imag, '-', color=color, lw=lw, alpha=alpha)

    ax.legend(
        handles=[
            matplotlib.patches.Patch(color='lightgreen', label='a = 0'),
            matplotlib.patches.Patch(color='r', label='b = π'),
            matplotlib.patches.Patch(color='purple', label='b = -π'),
        ],
        loc='lower left',
    )


if False:
    @interact(
        t=(0., 1., 0.01),
    )
    def f(
        t=1.,
    ):
        _, ax = plt.subplots(1, 1, figsize=(12, 8))
        render(ax, t * FRAMES)
        
PyplotRenderer(render, frame_count=FRAMES, width=800, height=800)

# TODO: highlight specific contours, e.g. a=1, b=pi

In [ ]:
FRAMES = 250
GRID_SIZE_X = 6
GRID_SIZE_Y = 6
GRID_MAJOR = 60
GRID_MINOR = 5
GRID_COUNT = GRID_MAJOR * GRID_MINOR + 1
MIN_SCALE = 0.7
MAX_SCALE = 60.

x_vals = np.linspace(-GRID_SIZE_X, GRID_SIZE_X, GRID_COUNT)
y_vals = np.linspace(-GRID_SIZE_Y, GRID_SIZE_Y, GRID_COUNT)
x_vals = np.where(np.isclose(x_vals, 0), np.zeros_like(x_vals), x_vals)
y_vals = np.where(np.isclose(y_vals, 0), np.zeros_like(y_vals), y_vals)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xy_grid = (x_grid + y_grid * 1j)
xy_vals = xy_grid.ravel()

def render(ax, frame_num):
    keyframe_1 = int(FRAMES * 0.00)
    keyframe0 = int(FRAMES * 0.04)
    keyframe1 = int(FRAMES * 0.46)
    keyframe2 = int(FRAMES * 0.54)
    keyframe3 = int(FRAMES * 0.96)
    keyframe4 = int(FRAMES * 1.00)
    keyframe0b = (keyframe1 - keyframe0) // 2 + keyframe0
    keyframe2b = (keyframe3 - keyframe2) // 2 + keyframe2
    if frame_num < keyframe0:
        t = 0.
    elif frame_num < keyframe0b:
        sub_t = (frame_num - keyframe0) / (keyframe0b - keyframe0)
        t = sub_t ** 2 / 2
    elif frame_num < keyframe1:
        sub_t = (frame_num - keyframe0b) / (keyframe1 - keyframe0b)
        t = 1 - (1. - sub_t) ** 2 / 2
    elif frame_num < keyframe2:
        t = 1.
    elif frame_num < keyframe2b:
        sub_t = (frame_num - keyframe2) / (keyframe2b - keyframe2)
        t = 1 - sub_t ** 2 / 2
    elif frame_num < keyframe3:
        sub_t = (frame_num - keyframe2b) / (keyframe3 - keyframe2b)
        t = (1. - sub_t) ** 2 / 2
    else:
        t = 0.

    z_grid = xy_grid ** (1 + 3 * t)
    z_grid_interp = z_grid
    z_vals_interp = z_grid_interp.ravel()
    ax.set_title('a+bi -> (a+bi)^4')
    scale = MIN_SCALE
    if MAX_SCALE - MIN_SCALE > 0.:
        scale *= np.exp(np.log(MAX_SCALE / MIN_SCALE) * t)
    ax.set_xlim(-10. * scale, 10. * scale)
    ax.set_ylim(-10. * scale, 10. * scale)
    ax.set_aspect('equal')
    ax.axhline(0, color='k', ls='--', lw=0.5)
    ax.axvline(0, color='k', ls='--', lw=0.5)
    if scale < 30:
        major_ticks = 10
    else:
        major_ticks = 100
    ax.xaxis.set_major_locator(MultipleLocator(major_ticks))
    ax.yaxis.set_major_locator(MultipleLocator(major_ticks))
    if scale < 6:
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.grid()
    ax.set_xlabel('real (a)')
    ax.set_ylabel('imaginary (bi)')
    
    z_points = z_grid_interp[::GRID_MINOR, ::GRID_MINOR].ravel()
    ax.plot(z_points.real, z_points.imag, 'o', ms=2, alpha=0.6)
    for i in range(0, z_grid.shape[0], GRID_MINOR):
        vals = z_grid_interp[i, :]
        y = xy_grid[i, 0].imag
        alpha = 0.5
        lw = 0.6
        color = 'b'
        if np.isclose(y, 1.):
            color = 'r'
            alpha = 1.
            lw = 1.5
        elif np.isclose(y, 2.):
            color = 'purple'
            alpha = 1.
            lw = 1.5
        ax.plot(vals.real, vals.imag, '-', color=color, lw=lw, alpha=alpha)

    for i in range(0, z_grid.shape[1], GRID_MINOR):
        vals = z_grid_interp[:, i]
        x = xy_grid[0, i].real
        alpha = 0.5
        lw = 0.6
        color = 'b'
        if np.isclose(x, 0.):
            color = 'lightgreen'
            alpha = 1.
            lw = 2
        ax.plot(vals.real, vals.imag, '-', color=color, lw=lw, alpha=alpha)

    ax.legend(
        handles=[
            #matplotlib.patches.Patch(color='lightgreen', label='a = 0'),
            #matplotlib.patches.Patch(color='r', label='b = π'),
            #matplotlib.patches.Patch(color='purple', label='b = -π'),
        ],
        loc='lower left',
    )


if False:
    @interact(
        t=(0., 1., 0.01),
    )
    def f(
        t=1.,
    ):
        _, ax = plt.subplots(1, 1, figsize=(12, 8))
        render(ax, t * FRAMES)
        
PyplotRenderer(render, frame_count=FRAMES, width=800, height=800)

# TODO: highlight specific contours, e.g. a=1, b=pi

In [ ]:
def quadratic_interpolate(t, t1, t2):
    t1 = max(min(t1, 1.), 0.)
    t2 = max(min(t2, 1.), 0)
    if t1 + t2 > 1.:
        norm = t1 + t2
        t1 /= norm
        t2 /= norm
    c = 1 / (t2 * (2 - t2 - t1)) if t2 > 0. else 0.
    a = 1 / (t1 * (2 - t2 - t1)) if t1 > 0. else 0.
    b = a * t1 ** 2
    m = (1 - c * t2 ** 2 - b) / (1 - t2 - t1) if t2 + t1 < 1. else 0.
    return np.piecewise(
        t,
        [
            t < 0.,
            (t >= 0.) & (t < t1),
            (t >= t1) & (t < 1. - t2),
            (t >= 1. - t2) & (t < 1.),
            t >= 1.,
        ],
        [
            lambda t: np.zeros_like(t),
            lambda t: a * t ** 2,
            lambda t: m * (t - t1) + b,
            lambda t: 1 - c * (1 - t) ** 2,
            lambda t: np.ones_like(t),
        ]
    )


@interact(
    t1=(0., 1., 0.01),
    t2=(0., 1., 0.01),
)
def run(
    t1=0.25,
    t2=0.25,
):
    t = np.linspace(-0.1, 1.1, 100)
    y = quadratic_interpolate(t, t1, t2)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.plot(t, y)
    ax.set_title('Quadratic animation interpolator')
    ax.axhline(0)
    ax.axhline(1)
    ax.axvline(0)
    ax.axvline(1)
    ax.axvline(t1, ls='--', lw=1, alpha=0.6)
    ax.axvline(1 - t2, ls='--', lw=1, alpha=0.6)
    ax.set_ylim(-0.1, 1.1)